In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

25/04/12 11:00:58 WARN Utils: Your hostname, edrik-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.0.48 instead (on interface wlp2s0)
25/04/12 11:00:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/edrik/.ivy2/cache
The jars for the packages stored in: /home/edrik/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8a8fb0d5-d565-4ca2-bb50-479f06960c51;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/edrik/dev/projeto-eng-dados/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.6.1/iceberg-spark-runtime-3.5_2.12-1.6.1.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1!iceberg-spark-runtime-3.5_2.12.jar (25022ms)
:: resolution report :: resolve 1141ms :: artifacts dl 25027ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   1   |   1   |   0   ||   1   |   1   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-8a8fb0d5-d565-4ca2-b

In [2]:
spark

In [3]:
spark.sql(
  """
  CREATE TABLE local.carro_iceberg (id INT, placa STRING) USING iceberg
  """
)

DataFrame[]

In [4]:
spark.sql("select * from local.carro_iceberg").show()

+---+-----+
| id|placa|
+---+-----+
+---+-----+



In [6]:
spark.sql("INSERT INTO local.carro_iceberg VALUES (1, 'XYZ1J34'), (2, 'RLC5B93'), (3, 'ABV1V23')")

DataFrame[]

In [7]:
spark.sql("select * from local.carro_iceberg").show()

+---+-------+
| id|  placa|
+---+-------+
|  1|XYZ1J34|
|  2|RLC5B93|
|  3|ABV1V23|
|  1|XYZ1J34|
|  2|RLC5B93|
|  3|ABV1V23|
+---+-------+



In [8]:
spark.sql(
    """
    alter table local.carro_iceberg add column marca STRING, modelo STRING, ano INT
    """
)

DataFrame[]

In [9]:
spark.sql(
    """
    select * from local.carro_iceberg
    """
).show()

+---+-------+-----+------+----+
| id|  placa|marca|modelo| ano|
+---+-------+-----+------+----+
|  1|XYZ1J34| NULL|  NULL|NULL|
|  2|RLC5B93| NULL|  NULL|NULL|
|  1|XYZ1J34| NULL|  NULL|NULL|
|  3|ABV1V23| NULL|  NULL|NULL|
|  2|RLC5B93| NULL|  NULL|NULL|
|  3|ABV1V23| NULL|  NULL|NULL|
+---+-------+-----+------+----+



In [10]:
spark.sql(
    """
    update local.carro_iceberg set marca = 'Renault', modelo = 'Sandero', ano = 2021 where id = 1
    """
)

DataFrame[]

In [11]:
spark.sql(
    """
    select * from local.carro_iceberg
    """
).show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  2|RLC5B93|   NULL|   NULL|NULL|
|  3|ABV1V23|   NULL|   NULL|NULL|
|  1|XYZ1J34|Renault|Sandero|2021|
|  1|XYZ1J34|Renault|Sandero|2021|
|  2|RLC5B93|   NULL|   NULL|NULL|
|  3|ABV1V23|   NULL|   NULL|NULL|
+---+-------+-------+-------+----+



In [12]:
spark.sql('update local.carro_iceberg set marca="GM", modelo="tracker", ano=2020 where id = 2    ')

DataFrame[]

In [13]:
spark.sql('update local.carro_iceberg set marca="Ford", modelo="EcoSport", ano=2022 where id = 3    ')

DataFrame[]

In [14]:
spark.sql('select * from local.carro_iceberg').show()

+---+-------+-------+--------+----+
| id|  placa|  marca|  modelo| ano|
+---+-------+-------+--------+----+
|  3|ABV1V23|   Ford|EcoSport|2022|
|  3|ABV1V23|   Ford|EcoSport|2022|
|  1|XYZ1J34|Renault| Sandero|2021|
|  1|XYZ1J34|Renault| Sandero|2021|
|  2|RLC5B93|     GM| tracker|2020|
|  2|RLC5B93|     GM| tracker|2020|
+---+-------+-------+--------+----+



In [15]:
spark.sql('delete from local.carro_iceberg where id = 3')

DataFrame[]

In [16]:
spark.sql('select * from local.carro_iceberg').show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  2|RLC5B93|     GM|tracker|2020|
|  2|RLC5B93|     GM|tracker|2020|
|  1|XYZ1J34|Renault|Sandero|2021|
|  1|XYZ1J34|Renault|Sandero|2021|
+---+-------+-------+-------+----+

